Anzeige der Temperaturdaten aus der Heizung

In [22]:
import sqlite3
import os, sys
import ipywidgets
import pandas as pd
import matplotlib.pyplot as plt
db_filename = 'Temperaturen.db'


def get_df(db_filename):
    db_file = db_filename
    if os.path.exists(os.path.join('/var/lib/grafana', db_filename)):
        db_file = ps.path.join('/var/lib/grafana', db_filename)
    
    try:
        con = sqlite3.connect(db_file)
        df = pd.read_sql_query("SELECT * from Temperaturen", con)
    except:
        print('Die Datenbank ' + db_file + ' ist nicht vorhanden')
        raise SystemExit("Stop right there!")
    return df
    

    
def plot_temp(start, dauer, kurven):

    data = df.tail(200)
    #data.info()
    # UnixTime 	AussenTemp 	VorlaufTemp 	RuecklaufTemp 	WohnzimmerTemp
    data.plot(x="UnixTime", y=kurven)
    #data    

# main
try:    
    df= get_df(db_filename)
except SystemExit as _e:
    print(_e)
    sys.exit(-1)
    
df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s')
df.set_index(['UnixTime'])
df['Aussen'] = str(df['AussenTemp'])
df['Wohnzimmer'] = str(df['WohnzimmerTemp'])

ipywidgets.interact(plot_temp, start= 1 , dauer = (1,100,1), kurven=["VorlaufTemp", "RuecklaufTemp", "WohnzimmerTemp", "AussenTemp"])

interactive(children=(IntSlider(value=1, description='start', max=3, min=-1), IntSlider(value=50, description=…

<function __main__.plot_temp(start, dauer, kurven)>